In [ ]:
import riiideducation
env = riiideducation.make_env()



In [ ]:
iter_test = env.iter_test()

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import riiideducation
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import os
path="/kaggle/input/riiid-test-answer-prediction/"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
dtype = {
    'timestamp':'int64',
    'content_type_id':'bool',
    'content_id':'int16',
    'answered_correctly':'int8',
    'prior_question_elapsed_time':'float32',
    'prior_question_had_explanation':'int8'
}
cols = [
    'timestamp',
    'content_type_id',
    'content_id',
    'answered_correctly',
    'prior_question_elapsed_time',
    'prior_question_had_explanation'
]
df_lect = pd.read_csv(path+'lectures.csv', sep=',')
df_q = pd.read_csv(path+'questions.csv', sep=',')
df_sub = pd.read_csv(path+'example_sample_submission.csv', sep=',')
df_test = pd.read_csv(path+'example_test.csv', sep=',')
df_train = pd.read_csv(path+'train.csv',sep=',',usecols=cols,dtype=dtype)
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train.info()

In [ ]:
df_q['tags'] = df_q['tags'].astype(str)

tags = [x.split() for x in df_q['tags'].values]
tags = [item for elem in tags for item in elem]
tags = set(tags)
tags_df = pd.DataFrame()
tags = list(tags)
correct = df_train[df_train.answered_correctly != -1].groupby(["content_id", 'answered_correctly'], as_index=False).size()
correct = correct.pivot(index= "content_id", columns='answered_correctly', values='size')
correct.columns = ['Wrong', 'Right']
correct = correct.fillna(0)
correct[['Wrong', 'Right']] = correct[['Wrong', 'Right']].astype(int)
df_q = df_q.merge(correct, left_on = "question_id", right_on = "content_id", how = "left")
for i in range(len(tags)):
    df = df_q[df_q.tags.str.contains(tags[i])].agg({'Wrong': ['sum'], 'Right': ['sum']})
    df['tag'] = tags[i]
    df = df.set_index('tag')
    tags_df = tags_df.append(df)

tags_df['Percent_correct'] = tags_df.Right/(tags_df.Right + tags_df.Wrong)
tags_df = tags_df.sort_values(by = "Percent_correct")

tags_df.head()

In [ ]:

df_train = df_train[df_train.content_type_id==False]
df_train[df_train.content_type_id==False]
del df_train['content_type_id']

In [ ]:
# counter=0
# for x,i in enumerate(A):
#     if len(i[1])>=1000:
#         A[x] = (A[x][0],A[x][1][:1200])
#     counter += len(A[x][1])
        
        
# sz = len(A)
# print(counter, sz)

In [ ]:
diff = 1/tags_df.Percent_correct
diff

In [ ]:

del df_q['question_id']
del df_q['part']
df_q['dif'] = df_q['Wrong']/(df_q['Right']+df_q['Wrong'])
df_q
df_q.groupby('bundle_id')['dif'].sum()

In [ ]:
df_lect

In [ ]:
del df_q['correct_answer']
del df_q['tags']
del df_q['Wrong']
del df_q['Right']
df_train = df_train.merge(df_q,how = 'inner',left_on='content_id',right_on='bundle_id')
del df_train['content_id']
answers = df_train['answered_correctly']
del df_train['answered_correctly']
del df_train['bundle_id']
df_train

In [ ]:
df_train = df_train.fillna(0)

In [ ]:
df_train
df_train['timestamp'] = df_train['timestamp'].astype('float32')
df_train['prior_question_had_explanation'] = df_train['prior_question_had_explanation'].astype('float32')
answers = answers.to_numpy(dtype='bool')  
answers = np.array([answers, ~answers],dtype='int8').transpose()

model.fit(df_train, answers, epochs=1)

In [ ]:
from sklearn import preprocessing
df_train = df_train.values
min_max_scaler = preprocessing.MinMaxScaler()
df_train = min_max_scaler.fit_transform(df_train)


In [ ]:
import tensorflow as tf
from tensorflow import keras
model = keras.Sequential()
act = 'relu'
model.add(keras.layers.Dense(25, input_dim=4, activation=act))

model.add(keras.layers.Dense(20, activation=act))
model.add(keras.layers.Dense(17, activation=act))
model.add(keras.layers.Dense(15, activation=act))

model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['AUC'])


In [ ]:
model.fit(df_train, answers, epochs=5, batch_size=10000,verbose=1,validation_split=0.3)
del df_train
del answers
model.save('model')

In [ ]:
def Predict(df_test, df_q, model):
    df_test = df_test.merge(df_q,how = 'inner',left_on='content_id',right_on='bundle_id')
    row_id = df_test.row_id
    df_test = df_test[['timestamp','prior_question_elapsed_time','prior_question_had_explanation','dif']]
    df_test['timestamp'] = df_test['timestamp'].astype('float32')
    df_test['prior_question_had_explanation'] = df_test['prior_question_had_explanation'].astype('float32')
    df_test = df_test.values
    min_max_scaler = preprocessing.MinMaxScaler()
    df_test = min_max_scaler.fit_transform(df_test)
    pred = model.predict(df_test)
    return pred.reshape((2,len(pred)))[0]
    
    

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[test_df['content_type_id']==0]
    test_df['answered_correctly'] = Predict(test_df,df_q,model)
    test_df = test_df[['row_id', 'answered_correctly']]
    env.predict(test_df)


In [ ]:
test_df = df_test[df_test['content_type_id']==0]
test_df['answered_correctly'] = Predict(test_df,df_q,model)
test_df = test_df[['row_id', 'answered_correctly']]
print(test_df)